<a href="https://colab.research.google.com/github/Richardxiao123/TW_stock_analysis/blob/main/%E5%9B%9E%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#載入所有股票代號

In [ ]:
import pandas as pd

stock_code =pd.read_csv("stock_code.csv")
stock_code=stock_code.drop(columns=['stock_code'])
stock_code = stock_code.reset_index().rename(columns={'index': 'stockprice'})

print(stock_code)




# 數據處理


In [ ]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

data_test = pd.DataFrame()


for a in stock_code['stockprice']:
    ticker =f"{a}.TW"
    data = yf.download(ticker,start='2024-2-19', end="2024-3-7")

    if data.empty:
        print(f"{ticker}.無資料跳過")
        continue

    # 進行數據標準化
    scaler = MinMaxScaler()
    data_temp = pd.DataFrame(index=data.index)
    data_temp['name'] = f"{a}.TW"

    for x in ['Close', 'High', 'Low', 'Open', 'Volume']:
        data_temp[x] = scaler.fit_transform(data[[x]])  # 線性變換 0~1

    # 10 天為一組轉到同一排
    new_data = {}
    new_data['name'] = [data_temp['name'].iloc[0]]

    for i in ['Close', 'High', 'Low', 'Open']:
        for y in range(10):
            new_data[f"{i}{y+1}"] = data_temp[i].iloc[y]

    new_data = pd.DataFrame(new_data)
    data_test = pd.concat([data_test, new_data], ignore_index=True)
    #print(new_data)

print(data_test)


In [ ]:
from google.colab import files
data_test.to_csv('data_run.csv')
files.download('data_run.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 開始預測

In [ ]:
import pickle
import pandas as pd

with open("rf_model.pkl", "rb") as f:
    rf_model = pickle.load(f)

#讀檔
data_run =pd.read_csv("data_run.csv")

X_test=data_run.drop(columns=['name']).drop(columns=['Unnamed: 0'])
name=data_run['name']
print(X_test)

y_pred = rf_model.predict(X_test)

print(y_pred)

count=0
buy_list=[]
for n in y_pred:
    if n == 5:
        print(name[count])
        buy_list.append(name[count])
        count+=1
    else:
          count+=1

print(buy_list)

# 回測數據

In [ ]:
import yfinance as yf

buy_list = ['1615.TW', '2059.TW', '2328.TW', '2379.TW', '2385.TW', '2387.TW', '6799.TW',
            '2392.TW', '2420.TW', '2455.TW', '2482.TW', '2491.TW', '2543.TW', '2603.TW',
            '2637.TW', '3004.TW', '3028.TW', '3501.TW', '3563.TW', '3588.TW', '3592.TW',
            '4562.TW', '4588.TW', '4912.TW', '4994.TW', '5269.TW', '6165.TW', '6176.TW',
            '6177.TW', '6196.TW', '6409.TW', '6412.TW', '6438.TW', '6491.TW', '6754.TW']

change_rate=[]
for ticker in buy_list:
        back_test = yf.download(ticker, start='2024-3-6', end="2024-3-7")
        back_test2 = yf.download(ticker, start='2024-3-21', end="2024-3-22")

        # 確認資料下載成功

        x = back_test['Close'].iloc[0].item()
        y = back_test2['Close'].iloc[0].item()
        z = y - x
        change_rate.append(z/x)
        print(f"{ticker}: 變動比例 = {z / x}")


print(change_rate)
print('報酬率',sum(change_rate))
amount = len(change_rate)
print('平均報酬率',sum(change_rate)/amount)